We will use "IMDB movie review sentiment classification dataset"

Dataset Description: https://keras.io/api/datasets/imdb/

This is a dataset of 25,000 movie reviews from IMDB, tagged by sentiment (positive/negative). The reviews have been preprocessed and each review is coded as a list of (whole) word indexes. For convenience, words are indexed by their overall frequency in the dataset, so that, for example, the integer "3" encodes the 3rd most frequent word in the data.

In [1]:
#!pip install keras

In [2]:
#!pip install tensorflow

In [3]:
import numpy
import keras
import pandas as pd
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, Dropout, Embedding, Conv1D, MaxPooling1D
from tensorflow.keras.preprocessing import sequence
from keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Flatten

# fix random seed for reproducibility
numpy.random.seed(7)


In [86]:
db=imdb.load_data()

In [148]:
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

In [149]:
len(X_train)

25000

In [150]:
y_train

array([1, 0, 0, ..., 0, 1, 0], dtype=int64)

In [151]:
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [152]:
X_train.shape

(25000, 500)

we will use the embedding layer which defines the first hidden layer of the network. it must specify 3 arguments:

input_dim: the size of the vocabulary in the text

output_dim: this is the size of the vector space in which each word will be immersed

input_legth: this is the size of the sequence, for example if your documents contain 100 words each then it is 100

In [10]:
# creating tyhe model 
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=6, batch_size=64)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 32)           160000    
                                                                 
 conv1d (Conv1D)             (None, 500, 32)           3104      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 250, 32)          0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 100)               53200     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 216,405
Trainable params: 216,405
Non-trainable params: 0
__________________________________________________

In [11]:
# evaluation
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 88.04%


## a simple example of the embedding layer

In [12]:
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']

In [13]:
labels = [1,1,1,1,1,0,0,0,0,0]

In [14]:
vocab_size = 50

In [15]:
encoded_docs = [one_hot(d, vocab_size) for d in docs]

In [16]:
print(encoded_docs)

[[40, 45], [31, 11], [9, 39], [5, 11], [47], [8], [2, 39], [14, 31], [2, 11], [9, 19, 45, 39]]


In [17]:
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[40 45  0  0]
 [31 11  0  0]
 [ 9 39  0  0]
 [ 5 11  0  0]
 [47  0  0  0]
 [ 8  0  0  0]
 [ 2 39  0  0]
 [14 31  0  0]
 [ 2 11  0  0]
 [ 9 19 45 39]]


We are now ready to define our Embedding layer as part of our model.

The embedding has a vocabulary of 50 and an entry length of 4. We will choose a small embedding space of 8 dimensions.

The model is a simple binary classification model. It is important to note that the output of the Embedding layer will be 4 vectors of 8 dimensions each, one for each word. We flatten it (the flatten layer) into a 32-element vector to pass it to the Dense output layer. 

In [18]:
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 4, 8)              400       
                                                                 
 flatten (Flatten)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
import numpy as np
labels = np.array(labels)
model.fit(padded_docs, labels, epochs=50, verbose=0)

In [20]:
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 80.000001


## To Do: 

1. Try the same thing on Google reviews dataset ( the file is given in the lab directory)
2. try to change the embedding representation using Glove and Skipgram 

In [175]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Embedding, Conv1D, MaxPooling1D
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import spacy
import gensim
df = pd.read_csv("reviews.csv")
print(df.shape)
df.head()

(15746, 11)


,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,sortOrder,appId
0,Andrew Thomas,https://lh3.googleusercontent.com/a-/AOh14GiHd...,Update: After getting a response from the deve...,1,21,4.17.0.3,2020-04-05 22:25:57,"According to our TOS, and the term you have ag...",2020-04-05 15:10:24,most_relevant,com.anydo
1,Craig Haines,https://lh3.googleusercontent.com/-hoe0kwSJgPQ...,Used it for a fair amount of time without any ...,1,11,4.17.0.3,2020-04-04 13:40:01,It sounds like you logged in with a different ...,2020-04-05 15:11:35,most_relevant,com.anydo
2,steven adkins,https://lh3.googleusercontent.com/a-/AOh14GiXw...,Your app sucks now!!!!! Used to be good but no...,1,17,4.17.0.3,2020-04-01 16:18:13,This sounds odd! We are not aware of any issue...,2020-04-02 16:05:56,most_relevant,com.anydo
3,Lars Panzerbjørn,https://lh3.googleusercontent.com/a-/AOh14Gg-h...,"It seems OK, but very basic. Recurring tasks n...",1,192,4.17.0.2,2020-03-12 08:17:34,We do offer this option as part of the Advance...,2020-03-15 06:20:13,most_relevant,com.anydo
4,Scott Prewitt,https://lh3.googleusercontent.com/-K-X1-YsVd6U...,Absolutely worthless. This app runs a prohibit...,1,42,4.17.0.2,2020-03-14 17:41:01,We're sorry you feel this way! 90% of the app ...,2020-03-15 23:45:51,most_relevant,com.anydo


In [176]:
def to_sentiment(score):
    score = int(score)
    if score <= 2:
        return 0
    else:
        return 1
df['sentiment'] = df.score.apply(to_sentiment)

df = df[['content', 'sentiment']]
df = df.reset_index(drop=True)
df

,content,sentiment
0,Update: After getting a response from the deve...,0
1,Used it for a fair amount of time without any ...,0
2,Your app sucks now!!!!! Used to be good but no...,0
3,"It seems OK, but very basic. Recurring tasks n...",0
4,Absolutely worthless. This app runs a prohibit...,0
...,...,...
15741,I believe that this is by far the best app wit...,1
15742,It sometimes crashes a lot!!,1
15743,Works well for what I need,1
15744,Love it.,1


In [177]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Calculate the maximum sequence length
max_sequence_length = max(len(content.split()) for content in df["content"])

# Tokenize the content
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(df["content"])
sequences = tokenizer.texts_to_sequences(df["content"])

# Pad sequences to the same length
content_embeddings = pad_sequences(sequences, maxlen=max_sequence_length)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(content_embeddings, df["sentiment"], test_size=0.2, random_state=42)

# Load pre-trained GloVe embeddings
glove_embeddings = {}
with open('C:\glove.6B.50d.txt', encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        glove_embeddings[word] = vector

# Create an embedding matrix
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, index in tokenizer.word_index.items():
    if index < num_words:
        embedding_vector = glove_embeddings.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

# Define the LSTM model


# creating tyhe model 
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(num_words, embedding_dim, weights=[embedding_matrix], input_length=max_sequence_length, trainable=False))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

print(model.summary())




# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the LSTM model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)


Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_23 (Embedding)    (None, 391, 50)           598500    
                                                                 
 conv1d_11 (Conv1D)          (None, 391, 32)           4832      
                                                                 
 max_pooling1d_11 (MaxPoolin  (None, 195, 32)          0         
 g1D)                                                            
                                                                 
 lstm_19 (LSTM)              (None, 128)               82432     
                                                                 
 dense_19 (Dense)            (None, 1)                 129       
                                                                 
Total params: 685,893
Trainable params: 87,393
Non-trainable params: 598,500
__________________________________________

### Word Embedding with Glove

In [170]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Calculate the maximum sequence length
max_sequence_length = max(len(content.split()) for content in df["content"])

# Tokenize the content
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(df["content"])
sequences = tokenizer.texts_to_sequences(df["content"])

# Pad sequences to the same length
content_embeddings = pad_sequences(sequences, maxlen=max_sequence_length)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(content_embeddings, df["sentiment"], test_size=0.2, random_state=42)

# Load pre-trained GloVe embeddings
glove_embeddings = {}
with open('C:\glove.6B.50d.txt', encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        glove_embeddings[word] = vector

# Create an embedding matrix
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, index in tokenizer.word_index.items():
    if index < num_words:
        embedding_vector = glove_embeddings.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

# Define the LSTM model
model = Sequential()
model.add(Embedding(num_words, embedding_dim, weights=[embedding_matrix], input_length=max_sequence_length, trainable=False))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the LSTM model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)


Epoch 1/10
394/394 [==============================] - 136s 340ms/step - loss: 0.5450 - accuracy: 0.7371 - val_loss: 0.5011 - val_accuracy: 0.7663
Epoch 2/10
394/394 [==============================] - 129s 329ms/step - loss: 0.4800 - accuracy: 0.7776 - val_loss: 0.4908 - val_accuracy: 0.7705
Epoch 3/10
394/394 [==============================] - 133s 337ms/step - loss: 0.4564 - accuracy: 0.7880 - val_loss: 0.4642 - val_accuracy: 0.7829
Epoch 4/10
394/394 [==============================] - 141s 358ms/step - loss: 0.4569 - accuracy: 0.7932 - val_loss: 0.5499 - val_accuracy: 0.7260
Epoch 5/10
394/394 [==============================] - 138s 351ms/step - loss: 0.4213 - accuracy: 0.8094 - val_loss: 0.4375 - val_accuracy: 0.8013
Epoch 6/10
394/394 [==============================] - 152s 386ms/step - loss: 0.4004 - accuracy: 0.8216 - val_loss: 0.4224 - val_accuracy: 0.8121
Epoch 7/10
394/394 [==============================] - 134s 340ms/step - loss: 0.3726 - accuracy: 0.8365 - val_loss: 0.4095 -

### Word Embedding with Skipgram

In [171]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from gensim.models import Word2Vec

# Calculate the maximum sequence length
max_sequence_length = max(len(content.split()) for content in df["content"])

# Tokenize the content
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(df["content"])
sequences = tokenizer.texts_to_sequences(df["content"])

# Pad sequences to the same length
content_embeddings = pad_sequences(sequences, maxlen=max_sequence_length)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(content_embeddings, df["sentiment"], test_size=0.2, random_state=42)

# Train Word2Vec model on the content
word2vec_model = Word2Vec(sentences=df["content"], vector_size=embedding_dim, window=5, min_count=1, sg=1)
word2vec_model.train(df["content"], total_examples=len(df["content"]), epochs=10)

# Create an embedding matrix
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, index in tokenizer.word_index.items():
    if index < num_words:
        if word in word2vec_model.wv:
            embedding_matrix[index] = word2vec_model.wv[word]

# Define the LSTM model
model = Sequential()
model.add(Embedding(num_words, embedding_dim, weights=[embedding_matrix], input_length=max_sequence_length, trainable=False))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the LSTM model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)


Epoch 1/10
394/394 [==============================] - 188s 469ms/step - loss: 0.6518 - accuracy: 0.6423 - val_loss: 0.6733 - val_accuracy: 0.6276
Epoch 2/10
394/394 [==============================] - 150s 381ms/step - loss: 0.6489 - accuracy: 0.6442 - val_loss: 0.6571 - val_accuracy: 0.6273
Epoch 3/10
394/394 [==============================] - 148s 376ms/step - loss: 0.6477 - accuracy: 0.6446 - val_loss: 0.6572 - val_accuracy: 0.6286
Epoch 4/10
394/394 [==============================] - 151s 383ms/step - loss: 0.6472 - accuracy: 0.6446 - val_loss: 0.6566 - val_accuracy: 0.6289
Epoch 5/10
394/394 [==============================] - 139s 353ms/step - loss: 0.6458 - accuracy: 0.6450 - val_loss: 0.6564 - val_accuracy: 0.6292
Epoch 6/10
394/394 [==============================] - 159s 404ms/step - loss: 0.6453 - accuracy: 0.6453 - val_loss: 0.6591 - val_accuracy: 0.6292
Epoch 7/10
394/394 [==============================] - 150s 381ms/step - loss: 0.6438 - accuracy: 0.6457 - val_loss: 0.6560 -

The Glove Embedding word model gives better results than SkipGram